In [1]:
!curl https://raw.githubusercontent.com/km1414/the-jokes/master/data/prepared_data.csv -s --create-dirs -o data/prepared_data.csv
!curl https://raw.githubusercontent.com/km1414/the-jokes/master/requirements.txt -s -o requirements.txt
!pip install -U -r requirements.txt -q
!nvidia-smi

Wed Oct 12 17:47:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained(
    "gpt2-medium",
    bos_token="<|startoftext|>",
    eos_token="<|endoftext|>",
    pad_token="<|pad|>",
)
# model_version = "gpt2-medium"
model_version = "gpt2-large"
model = GPT2LMHeadModel.from_pretrained(model_version).cpu()
model.resize_token_embeddings(len(tokenizer))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

Embedding(50259, 1280)

In [4]:
df = pd.read_csv("data/prepared_data.csv", encoding="utf8")
df = df.loc[df["joke"].str[:4] == "The "].reset_index(drop=True)
df["joke"] = df["joke"].str.replace("\n", " ").str.strip()
jokes = df["joke"]
jokes.head(10)

0    The Irishman's dilemma. Do you eat the potatoe...
1             The less hair I have The more head I get
2    The light above my desk is going out. I feel l...
3    The memory on my phone is FIFA'ed Sorry.....Co...
4    The best way to make people remember you? Borr...
5    The Professor and the Strip Joint A professor ...
6            The greatest joke of all time... Android.
7    The man who invented knock-knock jokes... He d...
8    The asian girl I'm playing chess against is re...
9    The gay lover of Emperor Palaptine ....is insi...
Name: joke, dtype: object

In [5]:
max_length = max([len(tokenizer.encode(description)) for description in jokes])
print("Jokes count:", len(jokes))
print("Max joke length:", max_length)

Jokes count: 8911
Max joke length: 167


In [6]:
class TheJokesDataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            encodings_dict = tokenizer(
                "<|startoftext|>" + txt + "<|endoftext|>",
                truncation=True,
                max_length=max_length,
                padding="max_length",
            )
            self.input_ids.append(torch.tensor(encodings_dict["input_ids"]))
            self.attn_masks.append(torch.tensor(encodings_dict["attention_mask"]))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [7]:
dataset = TheJokesDataset(jokes, tokenizer, max_length=max_length)
train_size = int(0.9 * len(dataset))
train_dataset, val_dataset = random_split(
    dataset, [train_size, len(dataset) - train_size]
)

In [8]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    logging_steps=100,
    save_steps=2000,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    warmup_steps=10,
    weight_decay=0.05,
    report_to=None,
)

In [9]:
Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=lambda data: {
        "input_ids": torch.stack([f[0] for f in data]),
        "attention_mask": torch.stack([f[1] for f in data]),
        "labels": torch.stack([f[0] for f in data]),
    },
).train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 8019
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 8019


Step,Training Loss
100,1.026200
200,0.661500
300,0.651300
400,0.633000
500,0.644800
600,0.607900
700,0.570400
800,0.777800
900,0.730400
1000,0.586000


Saving model checkpoint to ./results/checkpoint-2000
Configuration saved in ./results/checkpoint-2000/config.json
Model weights saved in ./results/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-4000
Configuration saved in ./results/checkpoint-4000/config.json
Model weights saved in ./results/checkpoint-4000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-6000
Configuration saved in ./results/checkpoint-6000/config.json
Model weights saved in ./results/checkpoint-6000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-8000
Configuration saved in ./results/checkpoint-8000/config.json
Model weights saved in ./results/checkpoint-8000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=8019, training_loss=0.6017948204865404, metrics={'train_runtime': 5301.8371, 'train_samples_per_second': 1.512, 'train_steps_per_second': 1.512, 'total_flos': 5691943782835200.0, 'train_loss': 0.6017948204865404, 'epoch': 1.0})

In [10]:
generated = tokenizer("<|startoftext|>The", return_tensors="pt").input_ids.cuda()

In [16]:
sample_outputs = model.generate(
    generated,
    do_sample=True,
    top_k=50,
    max_length=100,
    top_p=0.95,
    temperature=1.5,
    num_return_sequences=20,
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [17]:
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

0: The man that invented predictive text has died Unfortunately his funfair isn't coming on the 11th so there's no fun tomorrow I've gotta stick him down!
1: The hardest part of climbing the corporate ladder was getting out that ass!
2: The NSA is like the EU All Politics goes To Windows.
3: The worst type of letter to letter correspondence... A student writes out my name with two sets of eyes...
4: The reason they're calling the situation #BlackSaturday Is due to how deep the racism and animosity gets.
5: The most deadly weapon in a fantasy football game is going to be your opponent in a 1 on 3 scenario... In that 1 On 3 scenario... In that 1 on 3 scenario you do NOT use a weapon in this game, you rely on making the best decisions and building a perfect team of 4.1
6: The man who built Hovis must have been a genomist.
7: The Past and Present walked into a bar... It was quite tense.
8: The biggest problem now is not having to clean the house when guests show a sleeping, dead-face at yo